In [1]:
import pandas as pd
from datetime import datetime

In [11]:
# Function to generate evaporation.dat file from CSV (with user-specified column for evapotranspiration in mm)
def create_evaporation_dat(csv_file, evap_column, output_file):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Ensure that the Date column is in datetime format
    df['date'] = pd.to_datetime(df['date'])
    
    # Convert evapotranspiration from millimeters (mm) to meters (m)
    df['Evapotranspiration_m'] = df[evap_column] / 1000  # 1 mm = 0.001 m
    
    # Calculate evaporation and transpiration values
    df['Evaporation'] = 0.70 * df['Evapotranspiration_m']  # 70% of evapotranspiration
    df['Transpiration'] = 0.30 * df['Evapotranspiration_m']  # 30% of evapotranspiration
    root_zone_depth = 0.6  # Constant root zone depth in meters
    
    # Reference date (start date in the CSV)
    reference_date = df['date'].min()
    
    # Conversion factors
    time_conversion_factor = 86400  # 1 day = 86400 seconds
    flux_conversion_factor = 1.157407408e-5
    
    # Open the output .dat file for writing
    with open(output_file, 'w') as f:
        # Write the header (File type, Date, and conversion factors)
        f.write(f"2 1\n")
        f.write(f"{reference_date.strftime('%d.%m.%Y %H:%M:%S.%f')[:-3]} {time_conversion_factor} {flux_conversion_factor}\n")
        
        # Write the data
        for idx, row in df.iterrows():
            # Calculate time offset in days and convert to seconds
            time_offset_days = (row['date'] - reference_date).days
            time_offset_seconds = time_offset_days * time_conversion_factor
            
            # Write time offset, evaporation, transpiration, and root zone depth
            f.write(f"{time_offset_seconds} {row['Evaporation']} {row['Transpiration']} {root_zone_depth}\n")

In [12]:
# Example usage
csv_file = r'C:\Users\as2023\bwSyncShare\01_Analysis\02_Europe_Flood\03_CATFLOW\in\climate\testde_3.csv'
output_file = r'C:\Users\as2023\bwSyncShare\01_Analysis\02_Europe_Flood\03_CATFLOW\in\climate\evaporation_lstm.dat'
evap_column = 'evap_lstm_column'  # Specify the evapotranspiration column name
create_evaporation_dat(csv_file, evap_column, output_file)
